In [5]:
!pip install deepl

In [81]:
import deepl

deepl_api_key = "18f0e533-a2a0-453b-8c1e-d03d4a8cbce6:fx"
translator = deepl.Translator(deepl_api_key)

result = translator.translate_text("Hello world!", target_lang="FR")
print(result.text)

result = translator.translate_text("Hallo Welt!", target_lang="FR")
print(result.text)

Bonjour à tous !
Bonjour le monde !


In [83]:
import pandas as pd

# Lecture des datas
df = pd.read_csv("df_avec_categorie_part2.csv")  # Fichier contenant la partie 2 à traduire
df_test = pd.read_csv("df_p2.1.csv")             # Fichier contenant 10% de la partie 2 (je l'ai scindé de df_p2.1.csv à df_p2.10.csv pour mes tests)
df.head()

# On compte le nombre de caractères total
df["nb_caracteres"] = df["merged"].str.len()
print("Il y a ", df["nb_caracteres"].sum(),"caractères")

# Il y a 11 millions de caractères et la version gratuite de Deepl n'offre que 500 000 caractères
# En amont de l'utilisation de Deepl, il va falloir identifier la langue pour ne traduire que ce qui n'est pas en français
# En effet, il semble que l'identification de la langue via Deepl consomme le quota


Il y a  11192009.0 caractères


In [39]:
pip install langdetect

     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     ------------------------------------- 981.5/981.5 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993251 sha256=a530d800d31d4dfb95e5706df40ddb58d61f0be3a08bc3ca4ca1c074032ecd2d
  Stored in directory: c:\users\nine_\appdata\local\pip\cache\wheels\c1\67\88\e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect
Note: you may need to restart the kernel to use updated packages.


In [85]:
# Identification de la langue via langdetect
from langdetect import detect, DetectorFactory

DetectorFactory.seed = 42       # On fixe le seed pour avoir toujours le même résultat 
                                # En effet, il peut y avoir des résultats variables et aléatoires sur les textes trop courts entre 2 exécutions 

def detect_language(text):
    try:
        return detect(text)
    except:
        return "unknown"

df["langue"] = df["merged"].apply(detect_language)
df_test["langue"] = df_test["merged"].apply(detect_language)
df.head()


,productid,imageid,prdtypecode,merged,category_name,nb_caracteres,langue
0,3958459979,1269395457,2905,northgard svardilfari clan of the horse dlc je...,Jeux vidéos pour PC,2179.0,fr
1,3692448707,1235181500,1920,ssassy bouteille verre vin bar taie doreiller ...,Linge de maison,1077.0,fr
2,157202884,912888566,1300,micro mountain rally circuit voituresscalextric,Modélisme,47.0,ro
3,3798276065,1246307381,2582,serre polycarbonate anthracite pserre jardin p...,Mobilier de jardin,667.0,fr
4,3045352416,1283015575,10,lalcool action physiologique application thrap...,Livres occasion,54.0,fr


In [87]:
# Nombre de caractères pour les textes qui ne sont pas en français
carac_non_fr = df[df["langue"] != "fr"]["nb_caracteres"].sum()
print("Nombre total de caractères à traduire :", carac_non_fr)

# Sur les 11 millions de caractères de la colonne merged, il reste toujours 1.2 millions qui ne sont pas en français.
# C'est trop pour notre quota Deepl de 500 000 caractères.

Nombre total de caractères à traduire : 1273424.0


In [89]:
# Nous choisissons de tronquer les lignes avec trop caractères.
df["merged_tronq"] = df["merged"].apply(lambda x: x[:2000] if isinstance(x, str) else x)
carac_non_fr = df[df["langue"] != "fr"]["merged_tronq"].str.len().sum()
print("Nombre total de caractères à traduire :", carac_non_fr)

# Malgré le tronque à 2000 caractères, il reste quasiment autant de caractères à traduire...


Nombre total de caractères à traduire : 1230772.0


In [93]:
df_test["merged_tronq"] = df["merged"].apply(lambda x: x[:2000] if isinstance(x, str) else x)
df_test.head()

,productid,imageid,prdtypecode,merged,category_name,langue,merged_tronq
0,3958459979,1.269395e+09,2905.0,northgard svardilfari clan of the horse dlc je...,Jeux vidéos pour PC,fr,northgard svardilfari clan of the horse dlc je...
1,3692448707,1.235182e+09,1920.0,ssassy bouteille verre vin bar taie doreiller ...,Linge de maison,fr,ssassy bouteille verre vin bar taie doreiller ...
2,157202884,9.128886e+08,1300.0,micro mountain rally circuit voituresscalextric,Modélisme,ro,micro mountain rally circuit voituresscalextric
3,3798276065,1.246307e+09,2582.0,serre polycarbonate anthracite pserre jardin p...,Mobilier de jardin,fr,serre polycarbonate anthracite pserre jardin p...
4,3045352416,1.283016e+09,10.0,lalcool action physiologique application thrap...,Livres occasion,fr,lalcool action physiologique application thrap...


In [95]:
# Fonction pour n'envoyer à l'API Deepl que ce qui n'est pas en français
def translate_text(text, lang):
    if lang != "fr":  # Si la langue n'est pas français, on traduit
        try:
            return translator.translate_text(text, target_lang="FR").text
        except Exception as e:
            print(f"Erreur de traduction : {e}")
            return text  # En cas d'erreur, on retourne le texte original
    return text  # Si déjà en français, on retourne également le texte original


In [97]:
# Tests sur la 1ère et 3ème ligne
texte1 = df_test.iloc[0]["merged_tronq"]
langue1 = df_test.iloc[0]["langue"]

texte3 = df_test.iloc[2]["merged_tronq"]
langue3 = df_test.iloc[2]["langue"]

trad1 = translate_text(texte1, langue1)
trad3 = translate_text(texte3, langue3)

print(f"Texte original 1 : {texte1}")
print(f"Texte traduit 1 : {trad1}")
print("\n")
print(f"Texte original 1 : {texte3}")
print(f"Texte traduit 1 : {trad3}")


Texte original 1 : northgard svardilfari clan of the horse dlc jeu tlchargement div pstrongnote strong ncessite jeu original bnorthgardb fonctionner compte steam connexion internet ncessaires activer tlcharger utiliser produitp propos jeu pmen brok eitria deux vaillants frre clan cheval connu force calme compte certains meilleurs artisan mondep pbles bonus clanbp pce clan dbute plusieurs avantages p ullieitria brok deux chef guerre clan peuvent tre recruts non camp militaire forgelilices deux chef guerre peuvent battre extraire minerai forgerlilila forge vlund remplace forge habituellelililes villageois construisent rparent btiments plus viteliul plorsquil atteint certains seuils renomme clan cheval gagne amliorations doutil supplmentaires possibilit forger seconde relique dispose galement propre relique amliore rsistance chef guerre ainsi vitesse dextraction forge p caractristiques forge vlund eitria brok forge habituelle remplace forge vlund lamlioration btiment spcial augmente vites

In [99]:
# Traduction via Deepl sur la partie test (2700 premières lignes de la partie 2)
df_test["trad"] = df_test.apply(lambda x: translate_text(x["merged_tronq"], x["langue"]), axis=1)


In [101]:
# Enregistrement dans un nouveau fichier Excel
df_test.to_excel("df_p2.1_traduit.xlsx", index=False)